In [1]:
import pandas as pd

In [2]:
df_plus_minus = pd.read_csv("Sources_df/game_plus_minus.csv", sep=";")
df_game_history = pd.read_csv("Sources_df/game_history.csv", sep=";")

In [3]:
#Валидация данных, из таблицы +/-, проверка на пустые ID team, на кол-во игрков в одном событии, а также, чтобы было больше одной
#команды в одном событии
# Читаем данные из game_plus_minus.csv
game_plus_minus = df_plus_minus

def count_and_remove_incomplete_events(game_plus_minus):
    # Удаляем строки, в которых ID team равен 0
    game_plus_minus_cleaned = game_plus_minus[game_plus_minus['ID team'] != 0]
    
    # Считаем количество строк с не полными данными до удаления
    incomplete_events_before = len(game_plus_minus) - len(game_plus_minus_cleaned)
    
    # Сгруппируем данные по столбцам 'ID event' и 'ID team' и посчитаем количество строк в каждой группе
    event_team_row_counts = game_plus_minus_cleaned.groupby(['ID event', 'ID team']).size().reset_index(name='row_count')
    
    # Проверяем, если в одном событии количество разных команд не равно 2, то удаляем это событие
    event_teams_counts = event_team_row_counts.groupby('ID event').size()
    incomplete_event_ids = event_teams_counts[event_teams_counts != 2].index
    game_plus_minus_cleaned = game_plus_minus_cleaned[~game_plus_minus_cleaned['ID event'].isin(incomplete_event_ids)]
    
    # Проверяем, чтобы в одном событии в каждой команде было не менее трех игроков, если их меньше, то удаляем это событие
    event_player_counts = game_plus_minus_cleaned.groupby(['ID event', 'ID team'])['ID player'].nunique().reset_index(name='player_count')
    incomplete_event_ids = event_player_counts[event_player_counts['player_count'] < 3]['ID event'].unique()
    game_plus_minus_cleaned = game_plus_minus_cleaned[~game_plus_minus_cleaned['ID event'].isin(incomplete_event_ids)]
    
    # Выводим информацию о количестве удаленных строк и итоговой размерности таблицы
    removed_rows_count = len(game_plus_minus) - len(game_plus_minus_cleaned)
    print(f"Количество удаленных строк: {removed_rows_count}")
    print(f"Размерность таблицы после удаления неполных событий: {game_plus_minus_cleaned.shape}")
    
    # Считаем количество удаленных ID event и количество игр с неполными данными
    incomplete_events_after = len(incomplete_event_ids)
    incomplete_games = len(game_plus_minus_cleaned['ID game'].unique())
    
    # Выводим информацию о количестве удаленных ID event и количестве игр с неполными данными
    print(f"Количество событий с неполными данными: {incomplete_events_after}")
    print(f"Количество игр с неполными событиями: {incomplete_games}")
    
    # Считаем процент игр с неполными данными
    total_games = len(game_plus_minus['ID game'].unique())
    incomplete_games_percentage = (incomplete_games / total_games) * 100
    
    # Выводим процент игр с неполными данными
    print(f"Процент игр с неполными данными: {incomplete_games_percentage:.2f}%")
    
    return game_plus_minus_cleaned

# Применяем функцию к таблице game_plus_minus
game_plus_minus_cleaned = count_and_remove_incomplete_events(game_plus_minus)

df_plus_minus = game_plus_minus_cleaned
# Сохраняем полученные данные в новый файл
#game_plus_minus_cleaned.to_csv('Temp_df/game_plus_minus_d.csv', index=False)

Количество удаленных строк: 6705
Размерность таблицы после удаления неполных событий: (300748, 7)
Количество событий с неполными данными: 19
Количество игр с неполными событиями: 4790
Процент игр с неполными данными: 98.89%


In [4]:
# Создаем пустой DataFrame для хранения результатов
df_pm = pd.DataFrame(columns=['ID season', 'ID game', 'ID team', 'ID player', 'p/m'])

# Создаем пустой словарь для хранения информации о том, какие игроки уже были учтены в каждой игре
players_in_game = {}

# Создаем пустой словарь для хранения информации о командах каждого игрока и номере сезона
player_teams = {}

# Итерируемся по строкам в df_plus_minus
for idx, row in df_plus_minus.iterrows():
    game_id = row['ID game']
    team_id = row['ID team']
    player_id = row['ID player']
    season_id = row['ID season']
    scoring_team = row['the scoring team']
    pm = 1 if team_id == scoring_team else -1  # Плюс или минус в зависимости от команды, забившей гол
    
    # Проверяем, был ли игрок уже учтен в этой игре
    if (game_id, player_id) not in players_in_game:
        # Если игрока нет, добавляем его в словарь результатов
        players_in_game[(game_id, player_id)] = pm
        
    # Проверяем, есть ли уже информация о команде и сезоне для этого игрока
    if (game_id, player_id) not in player_teams:
        player_teams[(game_id, player_id)] = {'team_id': team_id, 'season_id': season_id, 'game_id': game_id}
 
    else:
        # Если игрок уже есть, обновляем его плюс-минус
        players_in_game[(game_id, player_id)] += pm

# Создаем список DataFrame для каждого игрока
dfs = []
for (game_id, player_id), pm in players_in_game.items():
    dfs.append(pd.DataFrame([{'ID season': player_teams[(game_id, player_id)]['season_id'], 
                              'ID game': game_id,
                              'ID team': player_teams[(game_id, player_id)]['team_id'],  
                              'ID player': player_id,
                              'p/m': pm}]))

# Объединяем все DataFrame в один
df_pm = pd.concat(dfs, ignore_index=True)

# Сохраняем результат в файл
df_pm.to_csv('Temp_df/plus_minus_player_game.csv', index=False)
